In [20]:
import pandas as pd
df = pd.read_csv(r'C:/Users/Ashwin/Desktop/processed_data-1.csv', parse_dates = ['Payment Date'], index_col = 'Payment Date')
df.head()

,Client Name,Legal Entity,Paid Amount,Payment Status
Payment Date,,,,
2019-01-07,jpmchase,citibanknanewyork,30028.000,resolved
2019-01-10,jpmchase,citibanknanewyork,40011.000,resolved
2019-01-17,jpmchase,citibanknanewyork,-40049.000,resolved
2019-01-23,jpmchase,citibanknanewyork,-30043.000,resolved
2019-01-28,hdfcbankltdmumbaiheadoffice,citibanknanewyork,489580.312,resolved


In [21]:
from datetime import date, timedelta
import calendar
start_date = date(2018,12,1)
num_months = (date.today().year - 2019)*12 + date.today().month 
data = []
for i in range(num_months):
    for client in df['Client Name'].unique():
        days_in_month = calendar.monthrange(start_date.year, start_date.month)[1]
        date_ = start_date + timedelta(days=days_in_month)
        if date_ < date.today():
            df1 = df[str(date_)[:-3]]
        else:
            break
        count = len(df1[(df1['Client Name']==client)&(df1['Payment Status']=='resolved')])
        total = len(df1[df1['Client Name']==client])
        row = [date_, client, count, total]
        data.append(row)
    start_date = date_
df2 = pd.DataFrame(data,columns = ['Payment Date','Client Name', 'Resolved', 'Total']).set_index('Payment Date')
df2

,Client Name,Resolved,Total
Payment Date,,,
2019-01-01,jpmchase,5,5
2019-01-01,hdfcbankltdmumbaiheadoffice,2,3
2019-01-01,barclays,0,0
2019-01-01,hsbc,0,0
2019-01-01,bankofbarodamumbaiheadoffice,0,0
...,...,...,...
2020-06-01,hdfcbankltdmumbaiheadoffice,7,9
2020-06-01,barclays,6,6
2020-06-01,hsbc,3,3


In [22]:
df=df2
df

,Client Name,Resolved,Total
Payment Date,,,
2019-01-01,jpmchase,5,5
2019-01-01,hdfcbankltdmumbaiheadoffice,2,3
2019-01-01,barclays,0,0
2019-01-01,hsbc,0,0
2019-01-01,bankofbarodamumbaiheadoffice,0,0
...,...,...,...
2020-06-01,hdfcbankltdmumbaiheadoffice,7,9
2020-06-01,barclays,6,6
2020-06-01,hsbc,3,3


In [26]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pickle
import numpy as np 
import pandas as pd 

allClients = ['sbi','hdfcbankltdmumbaiheadoffice', 'jpmchase', 'bankofbarodamumbaiheadoffice', 'hsbc','barclays']
total_resolved={}
total_transaction={}
predicted_resolved={}
predicted_total={}
percentage = {}
for clients in allClients :
    total_resolved[clients]=list()
    total_transaction[clients]=list()
for i in range(len(df)):
    total_resolved[df.iloc[i,0]].append(df.iloc[i,1])
    total_transaction[df.iloc[i,0]].append(df.iloc[i,2])
for client in allClients :
    model1 = ExponentialSmoothing(np.asarray(total_resolved[client]), trend='add', seasonal='add', seasonal_periods=6, damped=True)
    hw_model1 = model1.fit()
    model2 = ExponentialSmoothing(np.asarray(total_resolved[client]), trend='add', seasonal='add', seasonal_periods=6)
    hw_model2 = model2.fit()
    model = hw_model1 if hw_model1.aic < hw_model2.aic else hw_model2
    predicted_resolved[client]=model.forecast(6)
    
    model1 = ExponentialSmoothing(np.asarray(total_transaction[client]), trend='add', seasonal='add', seasonal_periods=6, damped=True)
    hw_model1 = model1.fit()
    model2 = ExponentialSmoothing(np.asarray(total_transaction[client]), trend='add', seasonal='add', seasonal_periods=6)
    hw_model2 = model2.fit()
    model = hw_model1 if hw_model1.aic < hw_model2.aic else hw_model2
    predicted_total[client]=model.forecast(6)
    
    num=0
    deno=0
    for i in range(6):
        num=abs(predicted_resolved[client][i])+num
        deno=abs(predicted_total[client][i])+deno
    if deno > 0:
        percentage[client]=(num*100)/deno
    else :
        percentage[client]=0
    
print(predicted_resolved)
print(predicted_total)
print(percentage)

F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle

{'sbi': array([-1.61889579, -2.04255859,  2.53504812,  0.17598749, -1.73855114,
       -0.92560908]), 'hdfcbankltdmumbaiheadoffice': array([14.98867595,  8.22259905, 17.88221629, 11.81895848, 10.71719278,
       12.17091829]), 'jpmchase': array([5.10425318, 3.22626924, 5.3319499 , 3.29309363, 0.60115328,
       2.25991353]), 'bankofbarodamumbaiheadoffice': array([ 6.97143932,  6.97939495, 10.92364161,  5.87086511,  3.87666558,
        6.94612285]), 'hsbc': array([2.40445104, 1.98952793, 3.52759565, 3.42028919, 2.4371499 ,
       3.84316225]), 'barclays': array([4.58683263, 5.57819255, 4.5642951 , 6.93933842, 4.52635543,
       6.69775035])}
{'sbi': array([-1.61889579, -2.04255859,  2.53504812,  0.17598749, -1.73855114,
       -0.92560908]), 'hdfcbankltdmumbaiheadoffice': array([15.13964867,  8.37913081, 18.04406473, 12.93325137, 10.83923923,
       12.3405124 ]), 'jpmchase': array([5.26294778, 3.29649309, 5.38059748, 3.40975062, 0.58887042,
       2.58540645]), 'bankofbarodamumbaiheado

F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
F:\PROGRAMS_F\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
